### Model Details

Details:
- It uses combined dataset (openface+marlin) for higher accuracy

Baseline:
- The baseline model is 66.5% test, 68.49% valid, 69.29% train+valid (Paper Dataset)
- Out baseline model is 74.36% test, 67.97% valid (Our Dataset)
  - Could be due to diff test, valid data

### Model Training

Accuracy (8 Oct 1am):
- Valid: 67.97%
- Test: 74.36%

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import normalize
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()

from keras.layers import Flatten, Concatenate, LSTM, Dense, Conv2D, Conv3D, GlobalAveragePooling1D, Dropout, MaxPooling2D

from tensorflow import keras
from tensorflow.keras import layers

from data_prep import data_loader_v1, data_loader_fusion
import config
import utils

In [4]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim
        )
        self.sequence_length = sequence_length
        self.output_dim = output_dim
        self.mask_layer = MaskComputationLayer()  # Use the new mask layer

    def call(self, inputs):
        # The inputs are of shape: `(batch_size, frames, num_features)`
        length = tf.shape(inputs)[1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_positions = self.position_embeddings(positions)
        return inputs + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return self.mask_layer(inputs)  # Use the mask layer to compute mask
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "sequence_length": self.sequence_length,
            "output_dim": self.output_dim
        })
        return config

class MaskComputationLayer(layers.Layer):
    def call(self, inputs):
        mask = tf.reduce_any(tf.cast(inputs, "bool"), axis=-1)
        return mask

In [5]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.3
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation=tf.nn.gelu), 
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]

        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "dense_dim": self.dense_dim,
            "num_heads": self.num_heads
        })
        return config

In [6]:
def build_user_model(x, input_shape, num_heads, mlp_units, mlp_dropout):
    dense_dim = 8
    embed_dim = input_shape[1]
    
    x = PositionalEmbedding(
        input_shape[0], embed_dim, name="frame_position_embedding_usr"
    )(x)
    
    
#     x = keras.layers.GlobalMaxPooling1D()(x)
    x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
    x = keras.layers.GlobalMaxPooling1D()(x)
    
#     # late fusin
#     for dim in mlp_units:
#         x = layers.Dense(dim, activation="relu")(x)
#         x = layers.Dropout(mlp_dropout)(x)
        
#     x = layers.Dense(4, activation='softmax')(x)
    return x

In [7]:
def build_stm_model(x, input_shape, num_heads, mlp_units, mlp_dropout):
    dense_dim = 8
    embed_dim = input_shape[1]
    
    x = PositionalEmbedding(
        input_shape[0], embed_dim, name="frame_position_embedding_stm"
    )(x)
    
  
    x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
    x = keras.layers.GlobalMaxPooling1D()(x)
    
#     # late fusing
#     for dim in mlp_units:
#         x = layers.Dense(dim, activation="relu")(x)
#         x = layers.Dropout(mlp_dropout)(x)
        
#     x = layers.Dense(4, activation='softmax')(x)
    return x

In [8]:
# def transformer_enc(
def transformer_encoder(inputs, head_size, num_heads, ff_dim=4, dropout=0.3):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs
    
    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [9]:
def build_model(
        input_shape_1,
        input_shape_2,
        head_size,
        num_heads,
        ff_dim,
        num_transformer_blocks,
        mlp_units,
        dropout=0,
        mlp_dropout=0,
        n_classes=4
    ):
    inputs_1 = keras.Input(shape=input_shape_1)
    inputs_2 = keras.Input(shape=input_shape_2)
    dense_dim = 8
    
    user_x = build_user_model(inputs_1, input_shape_1, num_heads, mlp_units, mlp_dropout)
    stm_x = build_stm_model(inputs_2, input_shape_2, num_heads, mlp_units, mlp_dropout)
    
    x = keras.layers.concatenate([user_x, stm_x])
    
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
        
    outputs = layers.Dense(n_classes, activation='softmax')(x)
    model = keras.Model([inputs_1, inputs_2], outputs)

    return model

In [10]:
BUFFER_SIZE = 100000

def make_ds(features, labels):
    ds = tf.data.Dataset.from_tensor_slices((features, labels))#.cache()
    ds = ds.shuffle(BUFFER_SIZE).repeat()
    return ds

def get_best_weights():
    import os
    def get_epoch(x):
        return int(x.split('epoch')[-1].split('-')[0])
    return sorted(os.listdir('./checkpoints/'), key=get_epoch)[-1]

def train(model_name, val=True):

    data= data_loader_fusion(model_name, val)

    if len(data) == 3:
        train, val, test = data
        val_x, val_y = val
        val_x1, val_x2 = val_x
    else:
        train, test = data
        
    train_x, train_y = train
    train_x1, train_x2 = train_x
    
    test_x, test_y = test
    test_x1, test_x2 = test_x
    
    total = train_y.shape[0]
    
    print ("train stats: ")
    print (train_x1.shape, train_y.shape)
    print (train_x2.shape, train_y.shape)
    
    if val:
        print ("val stats: ")
        print (val_x1.shape, val_y.shape)
        print (val_x2.shape, val_y.shape)
    
    print ("test stats: ")
    print (test_x1.shape, test_y.shape)
    print (test_x2.shape, test_y.shape)
    
    # class weight strategy
    class_weight = {
        k: (1 / train_y[train_y==k].shape[0]) * (total / 4) for k in np.unique(train_y)
    }
    
    
    BATCH_SIZE = 32

    input_shape_1 = train_x1.shape[1:]
    input_shape_2 = train_x2.shape[1:]
        
    model = build_model(
        input_shape_1,
        input_shape_2,
        head_size=256,
        num_heads=16,
        ff_dim=4,
        num_transformer_blocks=12,
        mlp_units=[512],
        mlp_dropout=0.3,
        dropout=0.3,
        n_classes=4
    )

    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=keras.optimizers.Adam(learning_rate=1e-4),
        metrics=["accuracy"],
    )
    
    filepath = "checkpoints/" + model_name + ".epoch{epoch:02d}-acc{val_accuracy:.2f}.keras"
    checkpoint = ModelCheckpoint(filepath=filepath, 
                             monitor='val_accuracy',
                             verbose=1, 
                             save_best_only=True,
                             mode='max')
    
    callbacks = [
                 keras.callbacks.EarlyStopping(patience=50, restore_best_weights=True),
                 checkpoint
    ]
    if not val:
        val_x1 = test_x1
        val_x2 = test_x2
        val_y = test_y
    model.fit([train_x1, train_x2], train_y, 
              validation_data=([val_x1, val_x2], val_y), 
              epochs=200, 
              callbacks=callbacks,
              )
    
    model.load_weights(f'checkpoints/{get_best_weights()}')
    print ("Evaluating on train set: ")
    model.evaluate([train_x1, train_x2], train_y)

    print ("Evaluating on valid set: ")
    model.evaluate([val_x1, val_x2], val_y)

    print ("Evaluating on test set: ")
    model.evaluate([test_x1, test_x2], test_y)

    print ("Classification report (val): ")
    print(classification_report(val_y, y_pred_val))
   
    print ("Classification report (test): ")
    print(classification_report(test_y, y_pred_test))

In [11]:
# import sys
import shutil, os
if __name__ == '__main__':
    shutil.rmtree('checkpoints/')
    os.mkdir('checkpoints')
    train(config.FUSION, val=True)

                                chunk           label
0   subject_68_0ng3yqwrg6_vid_0_0.mp4         Engaged
1   subject_68_0ng3yqwrg6_vid_0_1.mp4  Highly-Engaged
2  subject_68_0ng3yqwrg6_vid_0_10.mp4  Highly-Engaged
3  subject_68_0ng3yqwrg6_vid_0_11.mp4  Highly-Engaged
4  subject_68_0ng3yqwrg6_vid_0_12.mp4         Engaged
<class 'numpy.ndarray'> 7670
TrainXy: ['subject_68_0ng3yqwrg6_vid_0_0.mp4', 'subject_68_0ng3yqwrg6_vid_0_1.mp4', 'subject_68_0ng3yqwrg6_vid_0_10.mp4', 'subject_68_0ng3yqwrg6_vid_0_11.mp4', 'subject_68_0ng3yqwrg6_vid_0_12.mp4']
len TrainXy: 5752
TestXy: ['subject_96_dh18u00dyu_vid_0_0.mp4', 'subject_96_dh18u00dyu_vid_0_1.mp4', 'subject_96_dh18u00dyu_vid_0_10.mp4', 'subject_96_dh18u00dyu_vid_0_11.mp4', 'subject_96_dh18u00dyu_vid_0_12.mp4']
len TestXy: 1698
ValXy: ['subject_85_rda0o4n8zs_vid_0_0.mp4', 'subject_85_rda0o4n8zs_vid_0_1.mp4', 'subject_85_rda0o4n8zs_vid_0_10.mp4', 'subject_85_rda0o4n8zs_vid_0_11.mp4', 'subject_85_rda0o4n8zs_vid_0_12.mp4']
len ValXy: 892
not fo

C:\Python39\lib\site-packages\keras\src\layers\layer.py:391: UserWarning: `build()` was called on layer 'frame_position_embedding_usr', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
C:\Python39\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'transformer_encoder' (of type TransformerEncoder) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
C:\Python39\lib\site-packages\keras\src\layers\layer.py:391: UserWarning: `build()` was called on layer 'frame_position_embedding_stm', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. T

Epoch 1/200


C:\Python39\lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor', 'keras_tensor_1']. Received: the structure of inputs=('*', '*')
  warnings.warn(
C:\Python39\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
C:\Python39\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
C:\Python39\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a mask

171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5685 - loss: 1.1112
Epoch 1: val_accuracy improved from -inf to 0.56722, saving model to checkpoints/engage_gaze+hp+au_marlin.epoch01-acc0.57.keras
171/171 ━━━━━━━━━━━━━━━━━━━━ 386s 2s/step - accuracy: 0.5689 - loss: 1.1100 - val_accuracy: 0.5672 - val_loss: 0.9936
Epoch 2/200
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7085 - loss: 0.7038
Epoch 2: val_accuracy did not improve from 0.56722
171/171 ━━━━━━━━━━━━━━━━━━━━ 334s 2s/step - accuracy: 0.7086 - loss: 0.7037 - val_accuracy: 0.5401 - val_loss: 1.2309
Epoch 3/200
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7234 - loss: 0.6559
Epoch 3: val_accuracy improved from 0.56722 to 0.58608, saving model to checkpoints/engage_gaze+hp+au_marlin.epoch03-acc0.59.keras
171/171 ━━━━━━━━━━━━━━━━━━━━ 347s 2s/step - accuracy: 0.7235 - loss: 0.6557 - val_accuracy: 0.5861 - val_loss: 1.0445
Epoch 4/200
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7500 - loss: 0.5780
Epoc

NameError: name 'y_pred_val' is not defined